# Model Downloader (URLs only)

Dropdown + URL textbox + Download button, plus a batch URL downloader.

## Tokens (recommended)
- Set `CIVITAI_TOKEN` and `HF_TOKEN` as environment variables in RunPod.
- The downloader will automatically use `CIVITAI_TOKEN` for CivitAI URLs.
- For Hugging Face, direct `resolve/main/...` links usually work without a token unless gated.


In [ ]:
from pathlib import Path

# Update this if your ComfyUI lives elsewhere
COMFY = Path('/workspace/ComfyUI')
MODELS = COMFY / 'models'

folders = {
    'checkpoints': MODELS / 'checkpoints',
    'loras': MODELS / 'loras',
    'vae': MODELS / 'vae',
    'controlnet': MODELS / 'controlnet',
    'ipadapter': MODELS / 'ipadapter',
    'clip_vision': MODELS / 'clip_vision',
    'text_encoders': MODELS / 'text_encoders',
    'diffusion_models': MODELS / 'diffusion_models',
    'unet': MODELS / 'unet'
}

for p in folders.values():
    p.mkdir(parents=True, exist_ok=True)

folders

In [ ]:
!pip -q install "requests>=2.31" "tqdm>=4.66" "ipywidgets>=8.1"

In [ ]:
import os
print('CIVITAI_TOKEN set?', bool(os.environ.get('CIVITAI_TOKEN')))
print('HF_TOKEN set?', bool(os.environ.get('HF_TOKEN')))

In [ ]:
import re
import requests
from tqdm import tqdm

def _filename_from_cd(cd: str | None):
    if not cd:
        return None
    m = re.search(r'filename\\*?=(?:UTF-8\\'\\')?\"?([^\";]+)\"?', cd, flags=re.IGNORECASE)
    return m.group(1).strip() if m else None

def download(url: str, folder_key: str = 'checkpoints', filename: str | None = None, overwrite: bool = False):
    """Download a direct URL into ComfyUI/models/<folder_key> with resume support."""
    assert folder_key in folders, f"folder_key must be one of: {list(folders.keys())}"
    dest_dir = folders[folder_key]
    dest_dir.mkdir(parents=True, exist_ok=True)

    headers = {}
    civ_token = os.environ.get('CIVITAI_TOKEN')
    if 'civitai.com' in url and civ_token:
        headers['Authorization'] = f'Bearer {civ_token}'

    # First request: get filename + size
    with requests.get(url, headers=headers, stream=True, allow_redirects=True) as r:
        r.raise_for_status()
        if not filename:
            filename = _filename_from_cd(r.headers.get('content-disposition'))
        if not filename:
            filename = url.split('?')[0].rstrip('/').split('/')[-1] or 'download.bin'
        dest = dest_dir / filename

        if dest.exists() and not overwrite:
            print(f'Already exists: {dest}')
            return dest

        tmp = dest.with_suffix(dest.suffix + '.part')
        existing = tmp.stat().st_size if tmp.exists() else 0

        total = r.headers.get('content-length')
        total = int(total) + existing if total and existing else (int(total) if total else None)

    # Resume request
    if existing > 0:
        headers2 = dict(headers)
        headers2['Range'] = f'bytes={existing}-'
        rr = requests.get(url, headers=headers2, stream=True, allow_redirects=True)
        if rr.status_code not in (206, 200):
            rr.close()
            existing = 0
            rr = requests.get(url, headers=headers, stream=True, allow_redirects=True)
    else:
        rr = requests.get(url, headers=headers, stream=True, allow_redirects=True)

    with rr as r2:
        r2.raise_for_status()
        mode = 'ab' if existing > 0 else 'wb'
        with open(tmp, mode) as f, tqdm(total=total, initial=existing, unit='B', unit_scale=True, desc=filename) as pbar:
            for chunk in r2.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

    tmp.rename(dest)
    print(f'Saved: {dest}')
    return dest

## Single URL Downloader
Use this when you have one URL.

In [ ]:
import ipywidgets as w
from IPython.display import display, clear_output

folder_dd = w.Dropdown(
    options=list(folders.keys()),
    value='checkpoints',
    description='Folder:',
    layout=w.Layout(width='420px')
)

url_tb = w.Text(
    value='',
    placeholder='Paste a direct download URL (HF resolve/… or CivitAI api/download/…)',
    description='URL:',
    layout=w.Layout(width='900px')
)

fname_tb = w.Text(
    value='',
    placeholder='Optional: force filename (otherwise auto-detected)',
    description='Name:',
    layout=w.Layout(width='900px')
)

overwrite_cb = w.Checkbox(value=False, description='Overwrite if exists')

btn_dl = w.Button(description='Download (URL)', button_style='success')
btn_open = w.Button(description='List folder', button_style='info')

out = w.Output()

def _list_folder(_):
    p = folders[folder_dd.value]
    with out:
        clear_output(wait=True)
        print('Folder:', p)
        items = sorted(p.glob('*'))
        if not items:
            print('(empty)')
        else:
            for f in items[:80]:
                print(' -', f.name)
            if len(items) > 80:
                print(f'... ({len(items)-80} more)')

def _do_download(_):
    url = url_tb.value.strip()
    if not url:
        with out:
            clear_output(wait=True)
            print('Paste a URL first.')
        return
    name = fname_tb.value.strip() or None
    with out:
        clear_output(wait=True)
        print('Downloading to:', folders[folder_dd.value])
        print('URL:', url)
        download(url, folder_key=folder_dd.value, filename=name, overwrite=overwrite_cb.value)

btn_dl.on_click(_do_download)
btn_open.on_click(_list_folder)

display(w.VBox([
    w.HBox([folder_dd, overwrite_cb, btn_open]),
    url_tb,
    fname_tb,
    btn_dl,
    out
]))

## Batch URL Downloader
Paste multiple URLs (one per line) and download them all to the selected folder.

In [ ]:
batch_tb = w.Textarea(
    value='',
    placeholder='Paste one URL per line\n# comment lines are ignored\nhttps://...\nhttps://...\n',
    description='URLs:',
    layout=w.Layout(width='900px', height='180px')
)

btn_batch = w.Button(description='Download all (URLs)', button_style='success')
out2 = w.Output()

def _parse_urls(text: str):
    urls = []
    for line in text.splitlines():
        s = line.strip()
        if not s or s.startswith('#'):
            continue
        urls.append(s)
    return urls

def _batch(_):
    urls = _parse_urls(batch_tb.value)
    with out2:
        clear_output(wait=True)
        if not urls:
            print('Paste at least one URL (one per line).')
            return
        print(f'Downloading {len(urls)} file(s) to:', folders[folder_dd.value])
        failures = 0
        for i, url in enumerate(urls, start=1):
            try:
                print(f'\n[{i}/{len(urls)}] {url}')
                download(url, folder_key=folder_dd.value, filename=None, overwrite=overwrite_cb.value)
            except Exception as e:
                failures += 1
                print('FAILED:', e)
        print('\nDone. Failures:', failures)

btn_batch.on_click(_batch)

display(w.VBox([
    w.HBox([folder_dd, overwrite_cb]),
    batch_tb,
    btn_batch,
    out2
]))